## Import libraries

In [2]:
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt

C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\1479078273.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import data

In [3]:
df = pd.read_csv(r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\Sonorus - nuie data\Data\niue outcomes.csv")

In [4]:
df.head()

,Record_ID,File_name,Outcome
0,1,065150_M.WAV,0
1,1,064736_A.WAV,0
2,1,064926_P.WAV,0
3,1,065025_T.WAV,0
4,2,070521_M.WAV,0


In [5]:
df.tail()

,Record_ID,File_name,Outcome
855,90a,124458_P.WAV,1
856,90b,140843_A.WAV,1
857,90b,140936_P.WAV,1
858,90b,141122_M.WAV,1
859,90b,141031_T.WAV,1


In [6]:
print(df.iloc[804, 0], df.iloc[805, 0])
#the file loading method uptill 804 and after 804 index is different.
#a represents first recording, and b represents second recording. for some ids the data is recorded twice.

228 10a


## pad signal

In [7]:
def pad_signal(data, max_len):
    #pad_length - 1422490 = max_length of the signals in the dataset
    #pad_length after spectrogram = 355712 = max length of the spectrograms
    new_arr = np.zeros(max_len)
    if len(data) > max_len:
        new_arr = data[:max_len]
    else:
        new_arr[:len(data)] = data
    return new_arr

## Preprocess first recording

In [8]:
specs = []
outcome = []
for i in range(805): #

    ##get file path
    try: 
        file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-001\\Recor\\"
        file_path += df.loc[i, "Record_ID"] + "\\" + df.loc[i, "File_name"]

        #load file
        audio_file, sr = librosa.load(file_path, sr=None)
    except:
        file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
        file_path += df.loc[i, "Record_ID"] + "\\" + df.loc[i, "File_name"]

        #load file
        audio_file, sr = librosa.load(file_path, sr=None)

    
    #get 5 second segments of the audio
    for j in range(0, len(audio_file), 5*sr):
        segment = audio_file[j:j+5*sr]
        #padding - for the last segment since it might be less than 5 seconds
        padded_audio = pad_signal(segment, 5*sr)
        #spectrogram
        spec = librosa.feature.melspectrogram(y=padded_audio, sr=sr)
        specs.append(spec)

    Number_of_segments = len(audio_file)/(5*sr)
    for j in range(int(Number_of_segments)):
        #outcome
        if df.loc[i, "Outcome"] == 0: #normal
            outcome.append(np.array([1, 0])) #normal = 1, abnormal = 0  
        else:    
            outcome.append(np.array([0, 1])) #normal = 0, abnormal = 1
        

C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\3350238616.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
c:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [9]:
print(len(specs), len(outcome), specs[0].shape)

7245 7245 (128, 40)


## Preprocess second recording

In [10]:

for i in range(805, 860):
    #------a------------------
    if df.loc[i, "Record_ID"][-1] == 'a':
        try:
            file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-001\\Recor\\"
            file_path += df.loc[i, "Record_ID"][:-1] + "\\" + df.loc[i, "File_name"]

            #load file
            audio_file, sr = librosa.load(file_path, sr=None)
        except:
            file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
            file_path += df.loc[i, "Record_ID"][:-1] + "\\" + df.loc[i, "File_name"]

            #load file
            audio_file, sr = librosa.load(file_path, sr=None)

    #------b------------------
    else:
        try:
            #---------2nd scan under 002----------------
            file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
            file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "2nd scan" + "\\" + df.loc[i, "File_name"]

            #load file
            audio_file, sr = librosa.load(file_path, sr=None)
            #--------2nd test under 002----------------
        except:
            try:
                file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
                file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "2nd test" + "\\" + df.loc[i, "File_name"]

                #load file
                audio_file, sr = librosa.load(file_path, sr=None)
            
            except:
                try: 
                    #--------second test under 002----------------
                    file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
                    file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "second test" + "\\" + df.loc[i, "File_name"]

                    #load file
                    audio_file, sr = librosa.load(file_path, sr=None)
                    #--------2nd tets under 002----------------
                except:
                    try:
                        file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-002\\Recor\\"
                        file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "2nd tets" + "\\" + df.loc[i, "File_name"]

                        #load file
                        audio_file, sr = librosa.load(file_path, sr=None)
                    #--------2nd scan but under 001----------------
                    except:
                        try:
                            file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-001\\Recor\\"
                            file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "2nd scan" + "\\" + df.loc[i, "File_name"]

                            #load file
                            audio_file, sr = librosa.load(file_path, sr=None)
                        except:
                            
                            #--------2nd test but under 001----------------
                            file_path = "C:\\Users\\rishi\\OneDrive - Monash University\\Documents\\Monash\\Sonorus\\Sonorus - nuie data\\Data\\Recor-20240305T001003Z-001\\Recor\\"
                            file_path += df.loc[i, "Record_ID"][:-1] + "\\" + "2nd test" + "\\" + df.loc[i, "File_name"]

                            #load file
                            audio_file, sr = librosa.load(file_path, sr=None)
                            
    
    #get 5 second segments of the audio
    for j in range(0, len(audio_file), 5*sr):
        segment = audio_file[j:j+5*sr]
        #padding - for the last segment since it might be less than 5 seconds
        padded_audio = pad_signal(segment, 5*sr)
        #spectrogram
        spec = librosa.feature.melspectrogram(y=padded_audio, sr=sr)
        specs.append(spec)

    Number_of_segments = len(audio_file)/(5*sr)
    for j in range(int(Number_of_segments)):
        #outcome
        if df.loc[i, "Outcome"] == 0: #normal
            outcome.append(np.array([1, 0])) #normal = 1, abnormal = 0  
        else:    
            outcome.append(np.array([0, 1])) #normal = 0, abnormal = 1



C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:33: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:42: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:50: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:58: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(file_path, sr=None)
C:\Users\rishi\AppData\Local\Temp\ipykernel_11612\917655890.py:9: UserWarning: PySoundFile failed. Trying audi

In [11]:
print(len(specs), len(outcome), specs[0].shape)

7740 7740 (128, 40)


## Save

In [12]:
specs = np.array(specs)

In [13]:
specs.shape

(7740, 128, 40)

In [14]:
np.save(r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\Sonorus - nuie data\preprocessed data\specs segmented.npy", specs)

In [15]:
outcome = np.array(outcome)

In [16]:
outcome.shape

(7740, 2)

In [17]:
np.save(r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\Sonorus - nuie data\preprocessed data\outcome segmented.npy", outcome)